## Imports

In [ ]:
import os
import random
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter
from IPython.display import display, IFrame

from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode()

import mapboxgl.utils as mu
from mapboxgl.viz import CircleViz, HeatmapViz
token = os.environ['MAPBOX_ACCESS_TOKEN']

from exif import Image as eImage
from PIL import Image as pImage
from PIL import ImageEnhance

In [ ]:
album_folder = '/Users/matteoferla/Photos/somewhere'
album_name = 'Somewhere'

## Pandas of metadata

My camera has the following (`dir(img)`):

    ['_exif_ifd_pointer', '_gps_ifd_pointer', '_interoperability_ifd_Pointer', '_segments', 'artist', 'cfa_pattern', 'color_space', 'components_configuration', 'compressed_bits_per_pixel', 'compression', 'contrast', 'copyright', 'custom_rendered', 'datetime', 'datetime_digitized', 'datetime_original', 'exif_version', 'exposure_bias_value', 'exposure_mode', 'exposure_program', 'exposure_time', 'f_number', 'file_source', 'flash', 'flashpix_version', 'focal_length', 'focal_length_in_35mm_film', 'gain_control', 'get', 'get_file', 'gps_altitude', 'gps_altitude_ref', 'gps_datestamp', 'gps_latitude', 'gps_latitude_ref', 'gps_longitude', 'gps_longitude_ref', 'gps_map_datum', 'gps_satellites', 'gps_timestamp', 'gps_version_id', 'has_exif', 'jpeg_interchange_format', 'jpeg_interchange_format_length', 'light_source', 'make', 'maker_note', 'max_aperture_value', 'metering_mode', 'model', 'orientation', 'photographic_sensitivity', 'pixel_x_dimension', 'pixel_y_dimension', 'recommended_exposure_index', 'resolution_unit', 'saturation', 'scene_capture_type', 'scene_type', 'sensing_method', 'sensitivity_type', 'sharpness', 'software', 'subject_distance_range', 'subsec_time', 'subsec_time_digitized', 'subsec_time_original', 'user_comment', 'white_balance', 'x_resolution', 'y_and_c_positioning', 'y_resolution']

In [ ]:
dex = []


def get_metadata(file:str, folder:str):
    img = eImage(open(os.path.join(folder, file), 'rb'))
    d = dict(file=file,
            gps_version_id=img.gps_version_id,
            focal_length=img.focal_length,
            exposure_time=img.exposure_time,
            f_number=img.f_number,
             datetime=datetime.strptime(img.datetime, '%Y:%m:%d %H:%M:%S')
           )
    try:
        convert = lambda x: x[0]+x[1]/60+x[2]/3600
        d['gps_longitude']=convert(img.gps_longitude)
        d['gps_latitude']=convert(img.gps_latitude)
    except:
        pass
    return d
        
data = pd.DataFrame.from_records(
                                [get_metadata(file, album_folder) for file in os.listdir(album_folder) if '.jpg' in file.lower()]
                                )
display(data.head())
#data.to_pickle(album_name+'_photos.p')

### Map

In [ ]:
subdata = data.loc[~data.gps_longitude.isna()]

c = Counter([(round(row.gps_latitude, 3), -round(row.gps_longitude, 3)) for i, row in subdata.iterrows()]).most_common()
print('Most popular spot', c[0])

histo = pd.DataFrame.from_records(map(lambda d: {'lat': d[0][0], 'lon': d[0][1], 'counts': d[1]}, c))

print('Length', len(histo))
#display(histo)


####### Plot
points = mu.df_to_geojson(histo, lat='lat', lon='lon', precision=3)

## default way
breaks = mu.scale_between(0.1, c[0][1], 8)
## manual way
#breaks = [1, 2, 3, 4, 5, 20, 50, 80]
## fancy way
#breaks = [histo.counts.quantile(q=x*0.2) for x in range(0, 5)]

#Create a heatmap 
print('centre', histo.lat.mean(), histo.lon.mean())
HeatmapViz(points, 
           access_token=token,
           weight_property="counts",
           weight_stops=mu.create_weight_stops(breaks),
           color_stops=mu.create_color_stops(breaks, colors='Spectral'),
           radius_stops=mu.create_radius_stops(breaks, 1, 100),
           opacity=0.8,
           center=(histo.lon.mean(), histo.lat.mean()),
           zoom=6
          ).show()

Other graphs

In [ ]:
## FOCAL LENGTH HISTOGRAM

c = Counter(data['focal_length'])
ci = sorted(c.keys())
# Manual hack to determine lens by focal length
prime = [40]
ultra = [k for k in ci if k < 18]
kit = [k for k in ci if k > 16 and k != 40]

#plot
iplot({'data': [go.Bar(x=prime,
                       y=[c[i] for i in prime],
                       name='Prime lens',
                       marker_color='#ffdb58'
                          ),
                go.Bar(x=kit,
                       y=[c[i] for i in kit],
                       name='Kit lens',
                       marker_color='#ff7f50'
                          ),
                go.Bar(x=ultra,
                       y=[c[i] for i in ultra],
                       name='Ultrawide angle lens',
                       marker_color='#008080'
                          )
               ],
       'layout': dict(title='Focal length',
                      xaxis={'title': 'Focal length [mm]'},
                      yaxis={'title': 'N photos in 1 mm bin'},
                      bargap=0)
      })

In [ ]:
## Time of day

def hr2angle(hr):
    return (hr * 15) % 360

def hr_str(hr):
    # Normalize hr to be between 1 and 12
    hr_str = str(((hr-1) % 12) + 1)
    suffix = ' AM' if (hr % 24) < 12 else ' PM'
    return hr_str + suffix

h = data.datetime.apply(lambda x: x.hour)
c = Counter(h)

fig = go.Figure(data=
    go.Barpolar(
        r = list(c.values()),
        theta = [k * 360/24 for k in c.keys()]
    ))

fig.update_layout(showlegend=False,
                  polar=dict(angularaxis=dict(rotation=-90,
                                              direction="clockwise",
                                              tickvals = [hr2angle(hr) for hr in range(24)],
                                              ticktext = [hr_str(hr) for hr in range(24)])))

fig.show()

In [ ]:
## Exposure

c = Counter(data['exposure_time'])
keys = sorted(c.keys())
iplot({'data': [go.Scatter(x=[f'{round(k*10000)/10} ms' for k in keys],
                       y=[c[k] for k in keys],
                       mode='markers+lines'
                      )
               ],
       'layout': dict(title='Distribution of exposure time', xaxis=dict(title='Exposure time'), yaxis=dict(title='Counts'))
      })


Image merge. This is very very crude and could be done a lot better.
One issue is the orientation.

In [ ]:
n_in_collage = 10
n_collages = 10



for i in range(n_collages):
    mean = np.zeros((4000, 6000, 3))
    n = 0
    files = os.listdir(album_folder)
    random.shuffle(files)
    for file in files:
        if '.jpg' not in file.lower():
            continue
        print(file)
        # check orientation
        img_meta = eImage(open(os.path.join(album_folder, file), 'rb'))
        if str(img_meta.orientation) != 'Orientation.TOP_LEFT':
            continue
        # open
        img = pImage.open(os.path.join(album_folder, file))
        a = np.asarray(img)
        if a.shape[0] == 4000:
            mean = np.add(mean, a)
            n += 1
        if n > n_in_collage:
            break
    mean = (mean / n).astype(np.uint8)
    img = pImage.fromarray(mean)
    img = ImageEnhance.Color(img).enhance(10)
    img.show()
    img.save(f'collage_{i}.jpg')